In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_selection import RFE
rf = RandomForestClassifier()
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from scipy.stats import randint

In [ ]:
# data = pd.read_csv('~/Desktop/project_data_new/embedding_768_TCGA_COAD.csv')
# data.index = data['PatientID']
# drop the last two columns
# data = data.drop(data.columns[-2:], axis=1)

In [2]:
data = pd.read_csv('~/Desktop/project_data_new/embedding_768_TCGA_COAD_90percent_sample.csv')
data.index = data['PatientID']
# drop the 'PatientID' column
data = data.drop('PatientID', axis=1)

In [3]:
data_target = pd.read_csv('~/Desktop/project_data_new/target_768_avg_expanded.csv')

In [4]:
data_target.index = data_target['Unnamed: 0']
data_target = data_target.drop(['Unnamed: 0'], axis = 1)
# only keep the columns with category in the name
data_target = data_target.loc[:, data_target.columns.str.contains('category')]

In [5]:
data = data[data.index.isin(data_target.index)]

In [6]:
data_test = pd.read_csv('~/Desktop/project_data_new/embedding_768_TCGA_COAD_10percent_sample.csv')
data_test.index = data_test['PatientID']
# drop the 'PatientID' column
data_test = data_test.drop('PatientID', axis=1)
data_test = data_test[data_test.index.isin(data_target.index)]
data_test.shape

(42, 768)

In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.feature_selection import RFE
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score
# from scipy.stats import randint

# # Initialize the Random Forest Classifier
# rf = RandomForestClassifier(random_state=42)

# # Extract the feature values from data
# X = data.values

# # Set the chunk size (number of target variables per part)
# chunk_size = 40

# # Loop through target columns in chunks
# for chunk_start in range(120, 241, chunk_size):
#     # Get the current chunk of target columns
#     chunk_end = min(chunk_start + chunk_size, 241)
#     target_chunk = data_target.columns[chunk_start:chunk_end]
    
#     # Initialize list to store results for the current chunk
#     chunk_results = []

#     # Open a file to write classification reports for the current chunk
#     report_file_path = f"/home/qiuaodon/Desktop/CRC_image/Best_100_features_Randomforest_90percents/Classification_Reports_100features_part_{chunk_start}_{chunk_end}.txt"
#     with open(report_file_path, "w") as report_file:
#         #stratify 
#         for target_col in target_chunk:
#             # Extract the target column for the current category
#             Y = data_target[target_col].values

#             # Select top 100 features using RFE
#             rfe = RFE(rf, n_features_to_select=100)  # Choose top 100 features
#             X_selected = rfe.fit_transform(X, Y)
            
#             # Split the data into training and testing sets
#             #use stratify to cross validate
            
#             X_train, X_test, Y_train, Y_test = train_test_split(X_selected, Y, test_size=0.2, random_state=42)

#             # Define the hyperparameter distribution
#             param_dist = {
#                 'n_estimators': randint(100, 200),
#                 'max_depth': randint(20, 40),
#                 'min_samples_split': randint(10, 20),
#                 'min_samples_leaf': randint(1, 5),
#                 'max_features': ['sqrt', 'log2']
#             }

#             # Perform Randomized Search with cross-validation
#             random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=60, 
#                                                cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
#             random_search.fit(X_train, Y_train)

#             # Get the best model from Randomized Search
#             best_rf = random_search.best_estimator_
#             best_params = random_search.best_params_

#             # Fit the model on the training data for evaluation
#             best_rf.fit(X_train, Y_train)

#             # Evaluate on the training set
#             Y_train_pred = best_rf.predict(X_train)
#             train_report = classification_report(Y_train, Y_train_pred)
            
#             # Evaluate on the test set
#             Y_test_pred = best_rf.predict(X_test)
#             test_report = classification_report(Y_test, Y_test_pred, output_dict=True)

#             # Write classification reports to the single file
#             report_file.write(f"Classification Report for {target_col} (Train Set):\n")
#             report_file.write(train_report)
#             report_file.write("\n")
#             report_file.write(f"Classification Report for {target_col} (Test Set):\n")
#             report_file.write(classification_report(Y_test, Y_test_pred))
#             report_file.write("\n\n" + "="*80 + "\n\n")

#             # Calculate and store precision, recall, and accuracy
#             precision = precision_score(Y_test, Y_test_pred, average='weighted')
#             recall = recall_score(Y_test, Y_test_pred, average='weighted')
#             accuracy = accuracy_score(Y_test, Y_test_pred)

#             # Extract precision, recall, and f1-score specifically for class '1'
#             class_1_metrics = test_report.get('1', {"precision": None, "recall": None, "f1-score": None})

#             chunk_results.append({
#                 "Target Variable": target_col,
#                 "Precision": precision,
#                 "Recall": recall,
#                 "Accuracy": accuracy,
#                 "Class 1 Precision": class_1_metrics["precision"],
#                 "Class 1 Recall": class_1_metrics["recall"],
#                 "Class 1 F1-Score": class_1_metrics["f1-score"],
#                 "Best Hyperparameters": best_params
#             })
    
#     # Save the results for the current chunk to an Excel file
#     results_df = pd.DataFrame(chunk_results)
#     results_df.to_excel(f"/home/qiuaodon/Desktop/CRC_image/Best_features_REF/Precision_Recall_Accuracy_100features_part_{chunk_start}_{chunk_end}.xlsx", index=False)


Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits


In [8]:
# Align the targets for training and unseen test sets
data_target_train = data_target.loc[data.index]
data_target_unseen = data_target.loc[data_test.index]
data_target_train.shape

(407, 241)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score
from scipy.stats import randint

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Extract the feature values from data
X = data.values
X_test_unseen = data_test.values
# Set the chunk size (number of target variables per part)
chunk_size = 40

for chunk_start in range(200, 241, chunk_size):
    # Get the current chunk of target columns
    chunk_end = min(chunk_start + chunk_size, 241)
    target_chunk = data_target.columns[chunk_start:chunk_end]
    
    # Initialize list to store results for the current chunk
    chunk_results = []

    # Open a file to write classification reports for the current chunk
    report_file_path = f"/home/qiuaodon/Desktop/CRC_image/Best_100_features_Randomforest_90percents/Classification_Reports_100features_part_{chunk_start}_{chunk_end}.txt"
    with open(report_file_path, "w") as report_file:
        for target_col in target_chunk:
            # Extract the target column for the current category from training/validation set
            Y = data_target_train[target_col].values

            # Select top 100 features using RFE
            rfe = RFE(rf, n_features_to_select=100)  # Choose top 100 features
            X_selected = rfe.fit_transform(X, Y)

            # Split the remaining training/validation set into train and validation
            X_train, X_val, Y_train, Y_val = train_test_split(X_selected, Y, test_size=0.2, random_state=42, stratify=Y)

            # Define the hyperparameter distribution
            param_dist = {
                'n_estimators': randint(100, 200),
                'max_depth': randint(20, 40),
                'min_samples_split': randint(10, 20),
                'min_samples_leaf': randint(1, 5),
                'max_features': ['sqrt', 'log2']
            }

            # Perform Randomized Search with cross-validation
            random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=60, 
                                               cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
            random_search.fit(X_train, Y_train)

            # Get the best model from Randomized Search
            best_rf = random_search.best_estimator_
            best_params = random_search.best_params_

            # Fit the model on the training data for evaluation
            best_rf.fit(X_train, Y_train)

            # Evaluate on the training set
            Y_train_pred = best_rf.predict(X_train)
            train_report = classification_report(Y_train, Y_train_pred)
            
            # Evaluate on the validation set
            Y_val_pred = best_rf.predict(X_val)
            val_report = classification_report(Y_val, Y_val_pred, output_dict=True)

            # Test on the unseen test data
            X_test_unseen_selected = rfe.transform(X_test_unseen)  # Transform unseen test set using fitted RFE
            Y_test_unseen_col = data_target_unseen[target_col].values
            Y_test_pred = best_rf.predict(X_test_unseen_selected)
            unseen_test_report = classification_report(Y_test_unseen_col, Y_test_pred, output_dict=True)

            # Write classification reports to the single file
            report_file.write(f"Classification Report for {target_col} (Train Set):\n")
            report_file.write(train_report)
            report_file.write("\n")
            report_file.write(f"Classification Report for {target_col} (Validation Set):\n")
            report_file.write(classification_report(Y_val, Y_val_pred))
            report_file.write("\n")
            report_file.write(f"Classification Report for {target_col} (Unseen Test Set):\n")
            report_file.write(classification_report(Y_test_unseen_col, Y_test_pred))
            report_file.write("\n\n" + "="*80 + "\n\n")

            # Calculate and store metrics for validation set
            precision = precision_score(Y_val, Y_val_pred, average='weighted')
            recall = recall_score(Y_val, Y_val_pred, average='weighted')
            accuracy = accuracy_score(Y_val, Y_val_pred)

            # Extract validation Class 1 metrics
            class_1_metrics = val_report.get('1', {"precision": None, "recall": None, "f1-score": None})

            # Extract unseen test Class 1 metrics
            class_1_unseen_metrics = unseen_test_report.get('1', {"precision": None, "recall": None, "f1-score": None})

            # Calculate unseen test metrics
            unseen_precision = precision_score(Y_test_unseen_col, Y_test_pred, average='weighted')
            unseen_recall = recall_score(Y_test_unseen_col, Y_test_pred, average='weighted')
            unseen_accuracy = accuracy_score(Y_test_unseen_col, Y_test_pred)

            # Append results for the current target variable
            chunk_results.append({
                "Target Variable": target_col,
                "Validation Precision": precision,
                "Validation Recall": recall,
                "Validation Accuracy": accuracy,
                "Class 1 Precision (Validation)": class_1_metrics["precision"],
                "Class 1 Recall (Validation)": class_1_metrics["recall"],
                "Class 1 F1-Score (Validation)": class_1_metrics["f1-score"],
                "Unseen Test Precision": unseen_precision,
                "Unseen Test Recall": unseen_recall,
                "Unseen Test Accuracy": unseen_accuracy,
                "Class 1 Precision (Unseen Test)": class_1_unseen_metrics["precision"],
                "Class 1 Recall (Unseen Test)": class_1_unseen_metrics["recall"],
                "Class 1 F1-Score (Unseen Test)": class_1_unseen_metrics["f1-score"],
                "Best Hyperparameters": best_params
            })
    
    # Save the results for the current chunk to an Excel file
    results_df = pd.DataFrame(chunk_results)
    results_df.to_excel(f"/home/qiuaodon/Desktop/CRC_image/Best_100_features_Randomforest_90percents/Precision_Recall_Accuracy_100features_part_{chunk_start}_{chunk_end}.xlsx", index=False)


Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 5 folds for each of 60 candidates, totalling 300 fits
Fitting 

In [14]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score
from scipy.stats import randint

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Extract the feature values from data
X = data.values
# data_target only keep the index in data
data_target = data_target[data_target.index.isin(data.index)]
# Specify the single target variable
target_col = "category_stromal_34"

# Initialize a list to store results
results = []

# Set the path for the classification report file
report_file_path = f"/home/qiuaodon/Desktop/CRC_image/Best_100_features_Randomforest_90percents/Classification_Report_100features_{target_col}.txt"

# Extract the target column for the current category from training/validation set
Y = data_target[target_col].values
# Select top 100 features using RFE
# rfe = RFE(rf, n_features_to_select=100)  # Choose top 100 features
# X_selected = rfe.fit_transform(X, Y)
rf.fit(X, Y)
feature_importances = rf.feature_importances_
top_100_indices = np.argsort(feature_importances)[-100:]  # Indices of top 100 features
X_selected = X[:, top_100_indices] 
# Split the remaining training/validation set into train and validation
X_train, X_val, Y_train, Y_val = train_test_split(X_selected, Y, test_size=0.2, random_state=42, stratify=Y)

# Define the hyperparameter distribution
param_dist = {
    'n_estimators': randint(200, 1000),
    'max_depth': randint(30, 50),
    'min_samples_split': randint(10, 28),
    'min_samples_leaf': randint(1, 10),
    'max_features': ['sqrt', 'log2', None]
}

# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=200, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train, Y_train)

# Get the best model from Randomized Search
best_rf = random_search.best_estimator_
best_params = random_search.best_params_

# Fit the model on the training data for evaluation
best_rf.fit(X_train, Y_train)

# Evaluate on the training set
Y_train_pred = best_rf.predict(X_train)
train_report = classification_report(Y_train, Y_train_pred)

# Evaluate on the validation set
Y_val_pred = best_rf.predict(X_val)
val_report = classification_report(Y_val, Y_val_pred, output_dict=True)


# Write classification reports to a file
with open(report_file_path, "w") as report_file:
    report_file.write(f"Classification Report for {target_col} (Train Set):\n")
    report_file.write(train_report)
    report_file.write("\n")
    report_file.write(f"Classification Report for {target_col} (Validation Set):\n")
    report_file.write(classification_report(Y_val, Y_val_pred))
    report_file.write("\n")

# Calculate metrics for validation set
precision = precision_score(Y_val, Y_val_pred, average='weighted')
recall = recall_score(Y_val, Y_val_pred, average='weighted')
accuracy = accuracy_score(Y_val, Y_val_pred)

# Extract validation Class 1 metrics
class_1_metrics = val_report.get('1', {"precision": None, "recall": None, "f1-score": None})



# Append results for the target variable
results.append({
    "Target Variable": target_col,
    "Validation Precision": precision,
    "Validation Recall": recall,
    "Validation Accuracy": accuracy,
    "Class 1 Precision (Validation)": class_1_metrics["precision"],
    "Class 1 Recall (Validation)": class_1_metrics["recall"],
    "Class 1 F1-Score (Validation)": class_1_metrics["f1-score"],
    "Best Hyperparameters": best_params
})

# Save the results to an Excel file
results_df = pd.DataFrame(results)
results_df

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


,Target Variable,Validation Precision,Validation Recall,Validation Accuracy,Class 1 Precision (Validation),Class 1 Recall (Validation),Class 1 F1-Score (Validation),Best Hyperparameters
0,category_stromal_34,0.47627,0.487805,0.487805,0.466667,0.5,0.482759,"{'max_depth': 39, 'max_features': 'sqrt', 'min..."


In [57]:
best_params

{'max_depth': 40,
 'max_features': 'sqrt',
 'min_samples_leaf': 3,
 'min_samples_split': 10,
 'n_estimators': 616}

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score
from scipy.stats import randint

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Extract the feature values from data
X = data.values
X_test_unseen = data_test.values

# Specify the single target variable
target_col = "category_stromal_34"

# Initialize a list to store results
results = []

# Set the path for the classification report file
report_file_path = f"/home/qiuaodon/Desktop/CRC_image/Best_100_features_Randomforest_90percents/Classification_Report_100features_{target_col}.txt"

# Extract the target column for the current category from training/validation set
Y = data_target_train[target_col].values

# Select top 100 features using RFE
rfe = RFE(rf, n_features_to_select=100)  # Choose top 100 features
X_selected = rfe.fit_transform(X, Y)

# Split the remaining training/validation set into train and validation
X_train, X_val, Y_train, Y_val = train_test_split(X_selected, Y, test_size=0.2, random_state=42, stratify=Y)

# Define the hyperparameter distribution
param_dist = {
    'n_estimators': randint(100, 200),
    'max_depth': randint(20, 40),
    'min_samples_split': randint(10, 20),
    'min_samples_leaf': randint(1, 5),
    'max_features': ['sqrt', 'log2']
}

# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=200, 
                                   cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
random_search.fit(X_train, Y_train)

# Get the best model from Randomized Search
best_rf = random_search.best_estimator_
best_params = random_search.best_params_

# Fit the model on the training data for evaluation
best_rf.fit(X_train, Y_train)

# Evaluate on the training set
Y_train_pred = best_rf.predict(X_train)
train_report = classification_report(Y_train, Y_train_pred)

# Evaluate on the validation set
Y_val_pred = best_rf.predict(X_val)
val_report = classification_report(Y_val, Y_val_pred, output_dict=True)

# Test on the unseen test data
X_test_unseen_selected = rfe.transform(X_test_unseen)  # Transform unseen test set using fitted RFE
Y_test_unseen_col = data_target_unseen[target_col].values
Y_test_pred = best_rf.predict(X_test_unseen_selected)
unseen_test_report = classification_report(Y_test_unseen_col, Y_test_pred, output_dict=True)

# Write classification reports to a file
with open(report_file_path, "w") as report_file:
    report_file.write(f"Classification Report for {target_col} (Train Set):\n")
    report_file.write(train_report)
    report_file.write("\n")
    report_file.write(f"Classification Report for {target_col} (Validation Set):\n")
    report_file.write(classification_report(Y_val, Y_val_pred))
    report_file.write("\n")
    report_file.write(f"Classification Report for {target_col} (Unseen Test Set):\n")
    report_file.write(classification_report(Y_test_unseen_col, Y_test_pred))
    report_file.write("\n")

# Calculate metrics for validation set
precision = precision_score(Y_val, Y_val_pred, average='weighted')
recall = recall_score(Y_val, Y_val_pred, average='weighted')
accuracy = accuracy_score(Y_val, Y_val_pred)

# Extract validation Class 1 metrics
class_1_metrics = val_report.get('1', {"precision": None, "recall": None, "f1-score": None})

# Extract unseen test Class 1 metrics
class_1_unseen_metrics = unseen_test_report.get('1', {"precision": None, "recall": None, "f1-score": None})

# Calculate unseen test metrics
unseen_precision = precision_score(Y_test_unseen_col, Y_test_pred, average='weighted')
unseen_recall = recall_score(Y_test_unseen_col, Y_test_pred, average='weighted')
unseen_accuracy = accuracy_score(Y_test_unseen_col, Y_test_pred)

# Append results for the target variable
results.append({
    "Target Variable": target_col,
    "Validation Precision": precision,
    "Validation Recall": recall,
    "Validation Accuracy": accuracy,
    "Class 1 Precision (Validation)": class_1_metrics["precision"],
    "Class 1 Recall (Validation)": class_1_metrics["recall"],
    "Class 1 F1-Score (Validation)": class_1_metrics["f1-score"],
    "Unseen Test Precision": unseen_precision,
    "Unseen Test Recall": unseen_recall,
    "Unseen Test Accuracy": unseen_accuracy,
    "Class 1 Precision (Unseen Test)": class_1_unseen_metrics["precision"],
    "Class 1 Recall (Unseen Test)": class_1_unseen_metrics["recall"],
    "Class 1 F1-Score (Unseen Test)": class_1_unseen_metrics["f1-score"],
    "Best Hyperparameters": best_params
})

# Save the results to an Excel file
results_df = pd.DataFrame(results)
results_df

Fitting 5 folds for each of 200 candidates, totalling 1000 fits


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score

# Initialize the Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Extract the feature values from data
X = data.values
X_test_unseen = data_test.values

# Specify the single target variable
target_col = "category_stromal_34"

# Initialize a list to store results
results = []

# Set the path for the classification report file
report_file_path = f"/home/qiuaodon/Desktop/CRC_image/Best_100_features_Randomforest_90percents/Classification_Report_100features_{target_col}.txt"

# Extract the target column for the current category from training/validation set
Y = data_target_train[target_col].values

# Select top 100 features using RFE
rfe = RFE(rf, n_features_to_select=100)  # Choose top 100 features
X_selected = rfe.fit_transform(X, Y)

# Split the remaining training/validation set into train and validation
X_train, X_val, Y_train, Y_val = train_test_split(X_selected, Y, test_size=0.2, random_state=42, stratify=Y)

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 107, 150, 200],
    'max_depth': [20, 26, 30, 40],
    'min_samples_split': [10, 15, 20],
    'min_samples_leaf': [1, 2, 4, 5],
    'max_features': ['sqrt', 'log2']
}

# Perform Grid Search with cross-validation
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, 
                           scoring='f1_weighted', n_jobs=-1, verbose=1)
grid_search.fit(X_train, Y_train)

# Get the best model from Grid Search
best_rf = grid_search.best_estimator_
best_params = grid_search.best_params_

# Fit the model on the training data for evaluation
best_rf.fit(X_train, Y_train)

# Evaluate on the training set
Y_train_pred = best_rf.predict(X_train)
train_report = classification_report(Y_train, Y_train_pred)

# Evaluate on the validation set
Y_val_pred = best_rf.predict(X_val)
val_report = classification_report(Y_val, Y_val_pred, output_dict=True)

# Test on the unseen test data
X_test_unseen_selected = rfe.transform(X_test_unseen)  # Transform unseen test set using fitted RFE
Y_test_unseen_col = data_target_unseen[target_col].values
Y_test_pred = best_rf.predict(X_test_unseen_selected)
unseen_test_report = classification_report(Y_test_unseen_col, Y_test_pred, output_dict=True)

# Write classification reports to a file
with open(report_file_path, "w") as report_file:
    report_file.write(f"Classification Report for {target_col} (Train Set):\n")
    report_file.write(train_report)
    report_file.write("\n")
    report_file.write(f"Classification Report for {target_col} (Validation Set):\n")
    report_file.write(classification_report(Y_val, Y_val_pred))
    report_file.write("\n")
    report_file.write(f"Classification Report for {target_col} (Unseen Test Set):\n")
    report_file.write(classification_report(Y_test_unseen_col, Y_test_pred))
    report_file.write("\n")

# Calculate metrics for validation set
precision = precision_score(Y_val, Y_val_pred, average='weighted')
recall = recall_score(Y_val, Y_val_pred, average='weighted')
accuracy = accuracy_score(Y_val, Y_val_pred)

# Extract validation Class 1 metrics
class_1_metrics = val_report.get('1', {"precision": None, "recall": None, "f1-score": None})

# Extract unseen test Class 1 metrics
class_1_unseen_metrics = unseen_test_report.get('1', {"precision": None, "recall": None, "f1-score": None})

# Calculate unseen test metrics
unseen_precision = precision_score(Y_test_unseen_col, Y_test_pred, average='weighted')
unseen_recall = recall_score(Y_test_unseen_col, Y_test_pred, average='weighted')
unseen_accuracy = accuracy_score(Y_test_unseen_col, Y_test_pred)

# Append results for the target variable
results.append({
    "Target Variable": target_col,
    "Validation Precision": precision,
    "Validation Recall": recall,
    "Validation Accuracy": accuracy,
    "Class 1 Precision (Validation)": class_1_metrics["precision"],
    "Class 1 Recall (Validation)": class_1_metrics["recall"],
    "Class 1 F1-Score (Validation)": class_1_metrics["f1-score"],
    "Unseen Test Precision": unseen_precision,
    "Unseen Test Recall": unseen_recall,
    "Unseen Test Accuracy": unseen_accuracy,
    "Class 1 Precision (Unseen Test)": class_1_unseen_metrics["precision"],
    "Class 1 Recall (Unseen Test)": class_1_unseen_metrics["recall"],
    "Class 1 F1-Score (Unseen Test)": class_1_unseen_metrics["f1-score"],
    "Best Hyperparameters": best_params
})

# Save the results to an Excel file
results_df = pd.DataFrame(results)
results_df.to_excel(f"/home/qiuaodon/Desktop/CRC_image/Best_100_features_Randomforest_90percents/Precision_Recall_Accuracy_100features_{target_col}.xlsx", index=False)


Fitting 5 folds for each of 384 candidates, totalling 1920 fits


In [9]:
# import numpy as np
# import pandas as pd
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.feature_selection import RFE
# from sklearn.model_selection import train_test_split, RandomizedSearchCV
# from sklearn.metrics import classification_report, precision_score, recall_score, accuracy_score
# from scipy.stats import randint

# # Initialize the Random Forest Classifier
# rf = RandomForestClassifier(random_state=42)

# # Extract the feature values from data
# X = data.values

# # Initialize lists to store precision, recall, and accuracy for each target variable
# results = []

# # Open a single file to write all classification reports
# with open("/home/qiuaodon/Desktop/CRC_image/Best_features_REF/Classification_Reports_100features.txt", "w") as report_file:
    
#     # Loop through the first 120 target columns in data_target
#     for i, target_col in enumerate(data_target.columns[:241]):
#         # Extract the target column for the current category
#         Y = data_target[target_col].values

#         # Select top 100 features using RFE
#         rfe = RFE(rf, n_features_to_select=100)  # Choose top 100 features
#         X_selected = rfe.fit_transform(X, Y)
        
#         # Split the data into training and testing sets
#         X_train, X_test, Y_train, Y_test = train_test_split(X_selected, Y, test_size=0.2, random_state=42)

#         # Define the hyperparameter distribution (use distributions for random search)
#         param_dist = {
#             'n_estimators': randint(100, 200),
#             'max_depth': randint(20, 40),
#             'min_samples_split': randint(10, 20),
#             'min_samples_leaf': randint(1, 5),
#             'max_features': ['sqrt', 'log2']
#         }

#         # Perform Randomized Search with cross-validation
#         random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=60, 
#                                            cv=5, scoring='f1_weighted', n_jobs=-1, verbose=1, random_state=42)
#         random_search.fit(X_train, Y_train)

#         # Get the best model from Randomized Search
#         best_rf = random_search.best_estimator_
#         best_params = random_search.best_params_

#         # Fit the model on the training data for the final evaluation on train and test sets
#         best_rf.fit(X_train, Y_train)

#         # Evaluate on the training set
#         Y_train_pred = best_rf.predict(X_train)
#         train_report = classification_report(Y_train, Y_train_pred)
        
#         # Evaluate on the test set
#         Y_test_pred = best_rf.predict(X_test)
#         test_report = classification_report(Y_test, Y_test_pred, output_dict=True)

#         # Write classification reports to the single file
#         report_file.write(f"Classification Report for {target_col} (Train Set):\n")
#         report_file.write(train_report)
#         report_file.write("\n")
#         report_file.write(f"Classification Report for {target_col} (Test Set):\n")
#         report_file.write(classification_report(Y_test, Y_test_pred))
#         report_file.write("\n\n" + "="*80 + "\n\n")

#         # Calculate and store precision, recall, and accuracy
#         precision = precision_score(Y_test, Y_test_pred, average='weighted')
#         recall = recall_score(Y_test, Y_test_pred, average='weighted')
#         accuracy = accuracy_score(Y_test, Y_test_pred)

#         # Extract precision, recall, and f1-score specifically for class '1'
#         class_1_metrics = test_report.get('1', {"precision": None, "recall": None, "f1-score": None})

#         results.append({
#             "Target Variable": target_col,
#             "Precision": precision,
#             "Recall": recall,
#             "Accuracy": accuracy,
#             "Class 1 Precision": class_1_metrics["precision"],
#             "Class 1 Recall": class_1_metrics["recall"],
#             "Class 1 F1-Score": class_1_metrics["f1-score"],
#             "Best Hyperparameters": best_params
#         })

# # Save the results to an Excel file
# results_df = pd.DataFrame(results)
# results_df.to_excel("/home/qiuaodon/Desktop/CRC_image/Best_features_REF/Precision_Recall_Accuracy_100features.xlsx", index=False)


Fitting 5 folds for each of 60 candidates, totalling 300 fits


KeyboardInterrupt: 

# combine the results

In [2]:
import pandas as pd
import glob

# Specify the directory where your files are located
file_path_pattern = '/home/qiuaodon/Desktop/CRC_image/Best_100_features_Randomforest_90percents/Precision_Recall_Accuracy_100features_part_*.xlsx'

# Use glob to find all matching files
all_files = glob.glob(file_path_pattern)

# Create an empty list to store DataFrames
data_frames = []

# Loop through each file and append its DataFrame to the list
for file in all_files:
    df = pd.read_excel(file)
    data_frames.append(df)

# Concatenate all DataFrames
combined_df = pd.concat(data_frames, ignore_index=True)

# Save the combined DataFrame to a new Excel file
combined_df.to_excel('/home/qiuaodon/Desktop/CRC_image/Best_100_features_Randomforest_90percents/top_100features_Randomforest_241_targets_90train.xlsx', index=False)
combined_df


,Target Variable,Validation Precision,Validation Recall,Validation Accuracy,Class 1 Precision (Validation),Class 1 Recall (Validation),Class 1 F1-Score (Validation),Unseen Test Precision,Unseen Test Recall,Unseen Test Accuracy,Class 1 Precision (Unseen Test),Class 1 Recall (Unseen Test),Class 1 F1-Score (Unseen Test),Best Hyperparameters
0,category_tnk_1,0.476098,0.487805,0.487805,0.535714,0.555556,0.545455,0.371849,0.357143,0.357143,0.294118,0.357143,0.322581,"{'max_depth': 25, 'max_features': 'log2', 'min..."
1,category_tnk_2,0.514563,0.512195,0.512195,0.483871,0.555556,0.517241,0.543515,0.404762,0.404762,0.375000,0.400000,0.387097,"{'max_depth': 23, 'max_features': 'sqrt', 'min..."
2,category_tnk_3,0.483284,0.487805,0.487805,0.448276,0.464286,0.456140,0.180934,0.166667,0.166667,0.125000,0.181818,0.148148,"{'max_depth': 37, 'max_features': 'log2', 'min..."
3,category_tnk_4,0.560694,0.548780,0.548780,0.650000,0.500000,0.565217,0.588409,0.547619,0.547619,0.733333,0.523810,0.611111,"{'max_depth': 28, 'max_features': 'sqrt', 'min..."
4,category_tnk_5,0.546459,0.548780,0.548780,0.560000,0.518519,0.538462,0.353672,0.309524,0.309524,0.266667,0.285714,0.275862,"{'max_depth': 36, 'max_features': 'sqrt', 'min..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,category_stromal_10,0.456924,0.451220,0.451220,0.451613,0.500000,0.474576,0.480866,0.428571,0.428571,0.315789,0.600000,0.413793,"{'max_depth': 30, 'max_features': 'log2', 'min..."
237,category_stromal_11,0.501768,0.500000,0.500000,0.523810,0.407407,0.458333,0.367565,0.333333,0.333333,0.363636,0.266667,0.307692,"{'max_depth': 39, 'max_features': 'sqrt', 'min..."
238,category_stromal_12,0.426071,0.426829,0.426829,0.379310,0.407407,0.392857,0.379085,0.404762,0.404762,0.500000,0.500000,0.500000,"{'max_depth': 31, 'max_features': 'log2', 'min..."
239,category_stromal_13,0.565932,0.560976,0.560976,0.636364,0.500000,0.560000,0.391484,0.357143,0.357143,0.250000,0.363636,0.296296,"{'max_depth': 22, 'max_features': 'sqrt', 'min..."
